In [1]:
import pandas as pd


import numpy as np
import pandas as pd

import seaborn as sns

import matplotlib.pyplot as plt
%matplotlib inline

from sklearn import preprocessing

In [2]:
path = '/data/public/MLA/VCHAMPS-Quality/demographics_static_qual.csv'
demographics_static_test = pd.read_csv(path).iloc[:,1:]
demographics_static_test.head()

,Internalpatientid,Ethnicity,Gender,Races,Veteran flag
0,168899,Not specified,Male,Other,Yes
1,22214,Not Hispanic or Latino,Male,White,Yes
2,24552,Not Hispanic or Latino,Male,Black or African American,Yes
3,27878,Not specified,Male,Not specified (no value),Yes
4,29798,Not Hispanic or Latino,Male,White,Yes


In [3]:
demographics_static_test = demographics_static_test[demographics_static_test['Veteran flag'] == 'Yes']

In [4]:
pd.DataFrame(demographics_static_test['Races'].value_counts())

,Races
White,643
Not specified (no value),168
Black or African American,117
Other,55
White ; Other,7
Asian,5
Black or African American ; White,2
Black or African American ; Other,1


In [5]:
def races_combine(data):
    for i in range(len(data['Races'])):
        if  data['Races'].iloc[i] == 'White ; Other':
            data['Races'].iloc[i] = 'White'
        elif data['Races'].iloc[i] == 'Black or African American ; Other':
            data['Races'].iloc[i] = 'Black or African American'
        elif data['Races'].iloc[i] == 'Asian ; Other':
            data['Races'].iloc[i] = 'Asian'
        elif data['Races'].iloc[i] == '(Censored)':
            data['Races'].iloc[i] = 'Other'
        elif data['Races'].iloc[i] == 'Not specified (no value)':
            data['Races'].iloc[i] = 'Other'
        elif data['Races'].iloc[i] == 'Black or African American ; White':
            data['Races'].iloc[i] = 'Black or African American'
        elif data['Races'].iloc[i] == 'Black or African American ; Asian':
            data['Races'].iloc[i] = 'Black or African American'
        elif data['Races'].iloc[i] == 'White ; Asian':
            data['Races'].iloc[i] = 'White'
        elif data['Races'].iloc[i] == 'White ; Asian ; Other':
            data['Races'].iloc[i] = 'White'
        elif data['Races'].iloc[i] == 'Black or African American ; White ; Asian':
            data['Races'].iloc[i] = 'Black or African American'
        elif data['Races'].iloc[i] == 'Black or African American ; White ; Other':
            data['Races'].iloc[i] = 'Black or African American'
        
    return data



In [6]:
demographics_static_test2 = races_combine(demographics_static_test)
demographics_static_test2['Races'].value_counts()

/tmp/ipykernel_260757/2746446931.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['Races'].iloc[i] = 'Other'
/tmp/ipykernel_260757/2746446931.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['Races'].iloc[i] = 'White'
/tmp/ipykernel_260757/2746446931.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['Races'].iloc[i] = 'Black or African American'
/tmp/ipykernel_260757/2746446931.py:6: SettingWithCopyWarning: 
A 

White                        650
Other                        223
Black or African American    120
Asian                          5
Name: Races, dtype: int64

In [7]:
demographics_static_test3 = demographics_static_test2.copy()
demographics_static_test3['Ethnicity'].replace(['Hispanic or Latino','Not Hispanic or Latino','Not specified'],[0,1,2],inplace=True)
demographics_static_test3['Races'].replace(['White','Other','Black or African American', 'Asian'],[0,1,2,3],inplace=True)
demographics_static_test3['Gender'].replace(['Male','Female'],[1,0],inplace=True)

In [8]:
# One hot encoding races and ethnicity
data_dummy = pd.get_dummies(data = demographics_static_test3, columns = ['Ethnicity','Races'])
data_dummy = data_dummy[['Internalpatientid', 
       'Ethnicity_0', 'Ethnicity_1', 'Ethnicity_2', 'Races_0', 'Races_1',
       'Races_2', 'Races_3']]

In [9]:
demographics_static_test3 = pd.concat([demographics_static_test3,data_dummy.iloc[:,1:]],axis = 1)
demographics_static_test3

,Internalpatientid,Ethnicity,Gender,Races,Veteran flag,Ethnicity_0,Ethnicity_1,Ethnicity_2,Races_0,Races_1,Races_2,Races_3
0,168899,2,1,1,Yes,0,0,1,0,1,0,0
1,22214,1,1,0,Yes,0,1,0,1,0,0,0
2,24552,1,1,2,Yes,0,1,0,0,0,1,0
3,27878,2,1,1,Yes,0,0,1,0,1,0,0
4,29798,1,1,0,Yes,0,1,0,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...
995,164385,1,1,0,Yes,0,1,0,1,0,0,0
996,153684,0,1,0,Yes,1,0,0,1,0,0,0
997,167102,1,1,0,Yes,0,1,0,1,0,0,0
998,166856,1,1,0,Yes,0,1,0,1,0,0,0


In [10]:
demographics_static_test3 = demographics_static_test3.groupby('Internalpatientid').apply(lambda x:x)
demographics_static_test3 = demographics_static_test3.sort_values('Internalpatientid').reset_index(drop =True)
demographics_static_test3

/tmp/ipykernel_260757/3237507649.py:1: FutureWarning: Not prepending group keys to the result index of transform-like apply. In the future, the group keys will be included in the index, regardless of whether the applied function returns a like-indexed object.
To preserve the previous behavior, use

	>>> .groupby(..., group_keys=False)

To adopt the future behavior and silence this warning, use 

	>>> .groupby(..., group_keys=True)
  demographics_static_test3 = demographics_static_test3.groupby('Internalpatientid').apply(lambda x:x)


,Internalpatientid,Ethnicity,Gender,Races,Veteran flag,Ethnicity_0,Ethnicity_1,Ethnicity_2,Races_0,Races_1,Races_2,Races_3
0,67,1,1,0,Yes,0,1,0,1,0,0,0
1,200,1,1,0,Yes,0,1,0,1,0,0,0
2,291,2,1,1,Yes,0,0,1,0,1,0,0
3,330,1,1,0,Yes,0,1,0,1,0,0,0
4,351,1,1,0,Yes,0,1,0,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...
993,167907,1,1,0,Yes,0,1,0,1,0,0,0
994,167917,1,1,2,Yes,0,1,0,0,0,1,0
995,168008,1,1,2,Yes,0,1,0,0,0,1,0
996,168496,1,1,0,Yes,0,1,0,1,0,0,0


In [11]:
demographics_static_test3.to_csv('/home/daisy/FDA_Dataset/demographics_static_quality_admission.csv')